# Imports

Besides external libraries, we import several function from the local file : `utils.py`

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import brightway2 as bw

# Custom utils defined for inter-acv
from utils import *
from expression import *

# Init brightway2 and databases

In [2]:
# Setup bw2
bw.projects.set_current('B_Publication')
bw.bw2setup()

# Import Ecoinvent DB (if not already done)
# Update the PATH to suit your installation
importDb(ECOINVENT_DB_NAME, './ecoinvent 3.4_cutoff_ecoSpold02/datasets')

# We use a separate DB for defining our model, reset it beforehand
resetDb(ACV_DB_NAME)

# Parameters are stored at project level : reset them
resetParams()

Biosphere database already present!!! No setup is needed


Database 'ecoinvent 3.4 cut off' has already been imported 
Db incer-acv was here. Reseting it


# Define parameters

We define the parameters of the model.

The numeric parametes are instances of sympy' 'Symbol' . Thus, any python arithmetic expression composed of parameters will not result into direct evaluation, but the creation of a symbolic formula that can be manipulated later.

In [13]:
# Example 'float' parameter
a = newParamDef(
    'a', 
    ParamType.NUMBER, 
    default=0.5, 
    description="hello world")

b = newParamDef(
    'b', 
    ParamType.NUMBER, 
    default=0.5, 
    description="foo bar")

# Example 'enum' parameter, acting like a switch between several possibilities
# Enum parameters are not Symbol themselves
# They are a facility to represent many parameters at once '<paramName>_<enumValue>' 
# and should be used with the 'switch' method 
elec_switch_param = newParamDef(
    'elec_switch_param', 
    ParamType.ENUM, 
    values=["us", "eu"], 
    default="us", 
    description="Switch on electricty mix")

Exception: Param a already defined

# Get references to background activities and products

`utils` provide two functions for easy and fast search of activities in reference databases : 
* **findBioAct** : Search activity in **biosphere3** db
* **findTechAct** : Search activity in **ecoinvent** db


In [4]:
# Biosphere activities
ground_occupuation = findBioAct('Occupation, industrial area')
heat = findBioAct('Heat, waste', categories=['air'])

# Technosphere activities
alu = findTechAct("aluminium alloy production", "RER")

# Elec 
eu_elec = findTechAct("market group for electricity, medium voltage", 'ENTSO-E')
us_elec = findTechAct("market group for electricity, medium voltage", 'US')

# Define the model

The model is defined as a nested combination of reference activities and amount. 
Amounts are defined either as constant float values or algebric formulas implying the parameters defined above.

In [5]:
# Create new, parametrized activity

# You can create a virtual "switch" activity combining an enum param and several other activities 
elec_switch = switch(elec_switch_param, {
    "us" : us_elec,
    "eu" : eu_elec})

# Create a new activity
activity1 = newActivity(
    name="act1", unit="kg", # Any extra named arg will be added as activity attribute
    exchanges= { # We define exhanges as a dict of 'activity : amount'
        ground_occupuation:3 + a, # Amount can be a fixed value 
        heat: b + 0.2,
        elec_switch:4}) # Amount can be a Sympy expression (any arithmetic expression of Parameters)

# The final model is just the root activity referencing the others
model = newActivity("model", "kg", {
        activity1 : b * 5 + a + 1, # Reference the activity we just created
        heat: 3,
        alu:0.4 * a}) 

In [6]:
# Print_act displays activities as "pandas" tables
print_act(activity1) 
print_act(model)
          
# Note that symbolic expressions have not been evaluated at this stage

,amount /formula,unit,type
"'Occupation, industrial area' (square meter-year, None, ('natural resource', 'land'))",a + 3,square meter-year,biosphere
"'Heat, waste' (megajoule, None, ('air',))",b + 0.2,megajoule,biosphere
"'elec_switch_param switch' (kilowatt hour, GLO, None)",4,kilowatt hour,technosphere


,amount /formula,unit,type
"'act1' (kg, GLO, None)",a + 5*b + 1,kg,technosphere
"'Heat, waste' (megajoule, None, ('air',))",3,megajoule,biosphere
"'aluminium alloy production, AlMg3' (kilogram, RER, None)",0.4*a,kilogram,technosphere


# Select the impacts to consider

In [7]:
# List of impacts to consider
impacts = [m for m in bw.methods if 'ILCD 1.0.8 2016' in str(m) and 'no LT' in str(m)]

# Compute LCA

We provide two methods to compute LCA : 
* **multiLCA** : It uses brightway2 parametric capabilities. It is much slower and kept for comparing results.
* **multiLCAAlgebric** : It computes an algebric expression of the model and computes LCA once for all the background activities. Then it express each impact as a function of the parameter. This expression is compiled into 'numpy' native code, for fast computation on vectors of samples. This version is 1 million time faster.

In [8]:
# Uses brightway2 parameters
multiLCA(us_elec, impacts, 
                   
    # Parameters of the model
    a=1, 
    b=2, 
    elec_switch_param="us")

,"mineral, fossils and renewables",GWP 100a,freshwater and terrestrial acidification,freshwater ecotoxicity,freshwater eutrophication,ionising radiation,marine eutrophication,terrestrial eutrophication,carcinogenic effects,ionising radiation,non-carcinogenic effects,ozone layer depletion,photochemical ozone creation,"respiratory effects, inorganics",land use
379388913fee422a915764b6a021fd91,0.000003,0.662856,0.002418,0.195917,0.000079,6.013195e-07,0.000281,0.003751,5.199518e-09,0.077193,4.209638e-08,4.282936e-08,0.000901,0.000758,0.311929


In [9]:
# Compute with algebric implementation : the values should be the same
multiLCAAlgebric(
    elec_switch, # The model 
    impacts, # Impacts
    
    # Parameters of the model
    a=1, 
    b=2,
    elec_switch_param="us")

,"mineral, fossils and renewables",GWP 100a,freshwater and terrestrial acidification,freshwater ecotoxicity,freshwater eutrophication,ionising radiation,marine eutrophication,terrestrial eutrophication,carcinogenic effects,ionising radiation,non-carcinogenic effects,ozone layer depletion,photochemical ozone creation,"respiratory effects, inorganics",land use
0,0.000003,0.662856,0.002418,0.195917,0.000079,6.013195e-07,0.000281,0.003751,5.199518e-09,0.077193,4.209638e-08,4.282936e-08,0.000901,0.000758,0.311929


In [10]:
# Here is what the symbolic model looks like 
expr, _ = actToExpression(model)
expr

0.4*a*aluminium_alloy_production_almg3 + 3*heat_waste + (a + 5*b + 1)*(4*elec_switch_param_eu*market_group_for_electricity_medium_voltage1 + 4*elec_switch_param_us*market_group_for_electricity_medium_voltage + heat_waste*(b + 0.2) + occupation_industrial_area*(a + 3))

In [15]:
# Fast computation for millions of separate samples
multiLCAAlgebric(
    model, # The model 
    impacts, # Impacts
    
    # Parameters of the model
    a=list(range(1, 100000)), # LIst should be the same size for each param
    b=list(range(1, 100000)),
    elec_switch_param="eu")

,"mineral, fossils and renewables",GWP 100a,freshwater and terrestrial acidification,freshwater ecotoxicity,freshwater eutrophication,ionising radiation,marine eutrophication,terrestrial eutrophication,carcinogenic effects,ionising radiation,non-carcinogenic effects,ozone layer depletion,photochemical ozone creation,"respiratory effects, inorganics",land use
0,0.004127,1.428818e+01,0.090873,5.290014,0.001505,0.000015,0.009431,0.168374,2.147377e-07,1.891566,0.000002,0.000001,0.028514,0.008068,4.239340e+02
1,0.008239,2.686270e+01,0.170798,10.160906,0.002816,0.000029,0.017789,0.315800,4.157202e-07,3.521138,0.000004,0.000002,0.053895,0.015422,9.799657e+02
2,0.012351,3.943722e+01,0.250723,15.031798,0.004127,0.000042,0.026146,0.463226,6.167028e-07,5.150710,0.000005,0.000004,0.079277,0.022776,1.713597e+03
3,0.016463,5.201174e+01,0.330648,19.902690,0.005437,0.000055,0.034503,0.610652,8.176854e-07,6.780282,0.000007,0.000005,0.104659,0.030131,2.624829e+03
4,0.020575,6.458625e+01,0.410573,24.773582,0.006748,0.000068,0.042860,0.758078,1.018668e-06,8.409854,0.000009,0.000006,0.130041,0.037485,3.713661e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,411.199369,1.257391e+06,7992.114593,487065.281454,131.046152,1.320076,835.665241,14741.882529,2.009727e-02,162949.325875,0.172879,0.114795,2538.069502,735.407374,8.879402e+11
99995,411.203481,1.257403e+06,7992.194518,487070.152347,131.047463,1.320089,835.673599,14742.029955,2.009747e-02,162950.955447,0.172881,0.114796,2538.094884,735.414728,8.879579e+11
99996,411.207593,1.257416e+06,7992.274443,487075.023239,131.048773,1.320102,835.681956,14742.177381,2.009767e-02,162952.585019,0.172883,0.114797,2538.120266,735.422083,8.879757e+11
99997,411.211705,1.257428e+06,7992.354368,487079.894131,131.050084,1.320115,835.690313,14742.324807,2.009787e-02,162954.214591,0.172885,0.114798,2538.145648,735.429437,8.879935e+11
